In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Load and preprocess data
def load_data(path):
    data = pd.read_csv(path, sep='\t', header=None)
    data = data.dropna()
    # Format: Devanagari, Latin, Frequency
    return list(zip(data[1], data[0]))  # Latin input, Devanagari target

train_pairs = load_data('hi.translit.sampled.train.tsv')
val_pairs = load_data('hi.translit.sampled.dev.tsv')

# Prepare input and target sequences
input_texts = [inp for inp, _ in train_pairs]
target_texts = ['\t' + tgt + '\n' for _, tgt in train_pairs]

input_chars = sorted(set(''.join(input_texts)))
target_chars = sorted(set(''.join(target_texts)))

input_token_index = {char: i for i, char in enumerate(input_chars)}
target_token_index = {char: i for i, char in enumerate(target_chars)}

reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

# One-hot encoding
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, len(input_chars)), dtype="float32")
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, len(target_chars)), dtype="float32")
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, len(target_chars)), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# Define encoder
encoder_inputs = Input(shape=(None, len(input_chars)))
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define decoder
decoder_inputs = Input(shape=(None, len(target_chars)))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(target_chars), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=100,
    validation_split=0.2
)

# Inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Decode sequence
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, len(target_chars)))
    target_seq[0, 0, target_token_index['\t']] = 1.0

    decoded_sentence = ''
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n':
            break

        target_seq = np.zeros((1, 1, len(target_chars)))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]

    return decoded_sentence.strip()

# Encode text input
def encode_input_text(input_text):
    encoder_input = np.zeros((1, max_encoder_seq_length, len(input_chars)))
    for t, char in enumerate(input_text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.0
    return encoder_input

# Sample predictions
for seq_index in range(10):
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded = decode_sequence(input_seq)
    print(f"Input: {input_texts[seq_index]} → Prediction: {decoded} | Target: {target_texts[seq_index]}")

# Interactive prediction loop
while True:
    user_input = input("Enter a Latin word (or type 'exit' to quit): ").strip().lower()
    if user_input == 'exit':
        break
    encoded_input = encode_input_text(user_input)
    prediction = decode_sequence(encoded_input)
    print(f"Predicted Devanagari: {prediction}")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 26)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 65)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    289,792 │ input_layer[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    329,728 │ input_layer_1[0]… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 65)  │     16,705 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 636,225 (2.43 MB)

 Trainable params: 636,225 (2.43 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
553/553 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.0618 - loss: 1.1893 - val_accuracy: 0.0602 - val_loss: 1.2840
Epoch 2/100
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.0778 - loss: 1.1098 - val_accuracy: 0.0610 - val_loss: 1.2702
Epoch 3/100
553/553 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0944 - loss: 1.0539 - val_accuracy: 0.0604 - val_loss: 1.3103
Epoch 4/100
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1095 - loss: 1.0062 - val_accuracy: 0.0609 - val_loss: 1.3222
Epoch 5/100
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1152 - loss: 0.9835 - val_accuracy: 0.0607 - val_loss: 1.3336
Epoch 6/100
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1193 - loss: 0.9661 - val_accuracy: 0.0604 - val_loss: 1.3328
Epoch 7/100
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.1230 - loss: 0.9587 - val_accuracy: 0.0624 - val_loss: 1.3280
Epoch 8/100
553/553 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1261 - loss: 0.9445 - val_acc